In [243]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [244]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [242]:
spark.stop()

In [245]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.item

-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item


In [240]:
!pwd

/data/home/anna.egorova


In [246]:
rdd = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split('\t'))

In [247]:
rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [248]:
rdd_22 = rdd.filter(lambda x: x[1] == "302")

In [249]:
rdd_22.take(7)

[['186', '302', '3', '891717742'],
 ['191', '302', '4', '891560253'],
 ['49', '302', '4', '888065432'],
 ['54', '302', '4', '880928519'],
 ['62', '302', '3', '879371909'],
 ['206', '302', '5', '888180227'],
 ['197', '302', '3', '891409070']]

In [99]:
rdd_22.groupBy(lambda x: x[2]).take(5)

[('1', <pyspark.resultiterable.ResultIterable at 0x7fd2d70e6128>),
 ('4', <pyspark.resultiterable.ResultIterable at 0x7fd2d70e6518>),
 ('5', <pyspark.resultiterable.ResultIterable at 0x7fd2d70dfd68>),
 ('3', <pyspark.resultiterable.ResultIterable at 0x7fd2d70e6080>),
 ('2', <pyspark.resultiterable.ResultIterable at 0x7fd2d7073c50>)]

In [179]:
rdd_22.map(lambda x: (x[0], x[1], x[2], x[3])).reduceByKey(lambda a,b:a+b)

PythonRDD[547] at RDD at PythonRDD.scala:53

In [178]:
rdd_22.reduceByKey(lambda a,b:a+b)

PythonRDD[542] at RDD at PythonRDD.scala:53

In [250]:
df = rdd_22.toDF()
# df.show(10, 0)
df = df.groupBy("_3").count().orderBy("_3")
df.show(10, 0)

+---+-----+
|_3 |count|
+---+-----+
|1  |2    |
|2  |10   |
|3  |46   |
|4  |119  |
|5  |120  |
+---+-----+



In [251]:
list_df = df.select("count").rdd.flatMap(lambda x: x).collect()
print(f"hist_film: {list_df}")

hist_film: [2, 10, 46, 119, 120]


In [252]:
df_all = rdd.toDF()
# df.show(10, 0)
df_all = df_all.groupBy("_3").count().orderBy("_3")
df_all.show(10, 0)

+---+-----+
|_3 |count|
+---+-----+
|1  |6110 |
|2  |11370|
|3  |27145|
|4  |34174|
|5  |21201|
+---+-----+



In [253]:
list_df_all = df_all.select("count").rdd.flatMap(lambda x: x).collect()
print(f"hist_film: {list_df_all}")

hist_film: [6110, 11370, 27145, 34174, 21201]


In [ ]:
{
    "hist_film": [  
      5,
      14,
      46,
      98,
      134
   ],
   "hist_all": [  
      6110,
      11370,
      27145,
      34174,
      21201
   ]
}